In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil

root_dir = "drive/MyDrive/NewLungData"

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np

# Define data directory
data_dir = "drive/MyDrive/NewLungData"

# Define mean and standard deviation for normalization
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

# Define image size
image_size = (256, 256)

# Define batch size
batch_size = 32

# Define data generators for train, validation, and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir, 'train'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    os.path.join(data_dir, 'val'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
# Load pre-trained VGG16 model
base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))


In [ ]:
# Freeze convolutional base to prevent weights are being updated during training process
base_model.trainable = False

In [ ]:
# Base model architecture
base_model.summary()

In [ ]:
# Add custom classification layers
dropout_rate = 0.5
num_classes = 2

# Define the fully connected layers
classifier = tf.keras.Sequential([
    # Use GlobalAveragePooling2D instead of Flatten
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Concatenate the pre-trained convolutional base with the new classifier
model = tf.keras.Sequential([base_model, classifier])


In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate,weight_decay=1e-5),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
initial_epochs = 10
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(train_generator, epochs=initial_epochs, validation_data=val_generator,callbacks=[early_stopping])

In [ ]:
loss, accuracy = model.evaluate(test_generator)
print('Test accuracy: ', str(accuracy))

# Define the file path where you want to save the model
model_save_path =  "drive/MyDrive/NewLungData/tenserflowModels/vgg16"

In [ ]:
# No of layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
# Define grid search parameters for fine-tuning layers
fine_tune_layers= [15,10,5]

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


# Perform grid search for fine-tuning layers
results = []
for fine_tune_at in fine_tune_layers:
    print("-----------------------------------------------------")
    print("Freeze the layers before layer:", fine_tune_at)
    base_model.trainable = True

    # Freeze all the layers before the `fine_tune_at` layer
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False

    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])

    print("Trainable parameters:",trainable_params)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10,weight_decay=1e-5),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

    fine_tune_epochs = 10
    total_epochs =  initial_epochs + fine_tune_epochs

    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history_fine = model.fit(train_generator, epochs=total_epochs, initial_epoch=history.epoch[-1],validation_data=val_generator,callbacks=[early_stopping])

    loss, accuracy = model.evaluate(test_generator)
    print('Test accuracy of',fine_tune_at,"is : ", str(accuracy))

    # Define the file path where you want to save the model
    model_save_path =  "drive/MyDrive/NewLungData/tenserflowModels/vgg16_fn_"+str(no_of_fine_tune_layer)

    # Save the entire model to a single HDF5 file
    model.save(model_save_path)